# Import Dependencies

In [1]:
%pip install cdlib scikit-learn pandas plotly hvplot community python-louvain dynetx seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
import dynetx as dn

from cdlib import algorithms, evaluation
import csv
import numpy as np
import pandas as pd
import os
import seaborn as sns

from sklearn.metrics import rand_score
from sklearn.metrics import adjusted_rand_score


from scipy.stats import spearmanr
from scipy.stats import kendalltau
from scipy.spatial import distance

from networkx.algorithms import approximation
from networkx.algorithms.community.quality import modularity
from networkx.algorithms.centrality import closeness_centrality

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches

from sklearn.metrics import pair_confusion_matrix
import matplotlib.colors as mcolors
from matplotlib.patches import Patch

import colorsys
import random

Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'graph_tool', 'bayanpy', 'wurlitzer'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'leidenalg', 'infomap'}


# Level 1 Network Formation

We experiment with varying levels of delta, from (0.15, 0.5, 0.85).

In [ ]:
# Load the level 1 dataset
file_path = "../dataset/level_1/level_1.txt"
df = pd.read_csv(file_path, sep="\t")

level_2_df = pd.read_csv("../dataset/level_2/level_2.txt", sep="\t")

# Create a mapping of Gene to Level_2
gene_to_level_2 = dict(zip(level_2_df["Gene"], level_2_df["Level_2"]))

# Extract relevant columns
genes = df["Gene"]
level_1_classes = df["Level_1"]
expression_data = df.iloc[:, 2:].values  # Extract time-series expression data


# Compute Pearson correlation matrix
correlation_matrix = np.corrcoef(expression_data)

# Threshold for value-based graph construction
delta_values = [0.15, 0.5, 0.85]

# Initialize array to store results for each delta value
results = []

for delta in delta_values:
    threshold = delta
    adjacency_matrix = (correlation_matrix >= threshold ).astype(int)

    # Create a graph
    G = nx.Graph()

    # Add nodes with Level 1 classification as attributes
    for idx, gene in enumerate(genes):
        level_2_class = gene_to_level_2.get(gene, None)  # Get Level_2 if it exists, otherwise None
        G.add_node(gene, level_1=level_1_classes[idx], level_2=level_2_class)

    # Add edges based on Pearson correlation threshold
    edges = []
    for i in range(len(genes)):
        for j in range(i + 1, len(genes)):
            if adjacency_matrix[i, j] == 1:
                pearson_coefficient = correlation_matrix[i, j]
                G.add_edge(genes[i], genes[j], weight=pearson_coefficient)

    # Remove singletons
    singletons = list(nx.isolates(G))
    G.remove_nodes_from(singletons)

    # Compute additional graph properties
    avg_degree = sum(dict(G.degree()).values()) / len(G.nodes())  # Average degree
    density = nx.density(G)  # Graph density
    avg_clustering = nx.average_clustering(G)  # Average clustering coefficient
    largest_cc = max(nx.connected_components(G), key=len)  # Largest connected component
    largest_cc_size = len(largest_cc)  # Size of the largest component
    diameter = nx.diameter(G.subgraph(largest_cc)) if nx.is_connected(G) else None  # Diameter

    # Store results for the current delta, including the graph
    results.append({
        "Delta": delta,
        "Graph": G,  # Store the graph object
        "Number of Nodes": len(G.nodes()),
        "Number of Edges": G.number_of_edges(),
        "Transitivity": nx.transitivity(G) if len(G.nodes()) > 0 else 0,
        "Singletons Removed": len(singletons),
        "Average Degree": avg_degree,
        "Density": density,
        "Average Clustering Coefficient": avg_clustering,
        "Largest Component Size": largest_cc_size,
        "Diameter (if connected)": diameter
    })

# Convert results to a DataFrame for visualization (excluding the graph object)
graph_properties = pd.DataFrame([{k: v for k, v in result.items() if k != "Graph"} for result in results])

output_dir = "../results/level_1"
if not os.path.exists(output_dir):
	os.makedirs(output_dir)

# Save graph properties to a CSV file
graph_properties.to_csv(f"{output_dir}/graph_properties.csv", index=False)
graph_properties

# Level 1 Network Visualization

In [ ]:
TABLEAU_COLORS = {
    'tab:blue': '#1f77b4',
    'tab:pink': '#ff91d7',
    'tab:green': '#2ca02c',
    'tab:red': '#d62728',
    'tab:yellow': '#fff06b',
}
# Define unique colors for each Level 1 classification
unique_classes = level_1_classes.unique()
color_map = {cls: TABLEAU_COLORS[list(TABLEAU_COLORS.keys())[i]] for i, cls in enumerate(unique_classes)}

# Use the graph with delta = 0.85
G = results[2]["Graph"]  # Choose the graph with delta = 0.85

# Assign colors to nodes based on Level 1 classification
node_colors = [color_map[G.nodes[n]["level_1"]] for n in G.nodes()]

# Edge weights for visualization (stronger edges appear tighter)
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]

# Generate graph layout
plt.figure(figsize=(10, 10))
cell_cycle_phases = ["Early G1 Phase", "Late G1 Phase", "S Phase", "G2 Phase", "M Phase"]
legend_handles = [Patch(color=color, label=f'{cell_cycle_phases[cls-1]}') for cls, color in color_map.items()]
plt.legend(handles=legend_handles, loc="best", fontsize=8, title="Level 1 Classification")
pos = nx.spring_layout(G, weight='weight', k=0.1, seed=10)  # Spring layout based on weights

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=200, alpha=1, edgecolors="black")

# Draw edges with thickness based on weight
nx.draw_networkx_edges(G, pos, alpha=0.25, edge_color="black", width=[w * 2 for w in edge_weights])

# Show plot
# plt.title(f"Cho Gene Co-Expression Network (Value-Based, δ = {delta})")
plt.axis("off")
plt.savefig("../results/level_1/visualization/level1_gcn.png", dpi=300)
plt.show()

# Level 2 Network Visualization
For this section, nodes/genes in the dataset that do not have a Level 2 classification will be colored gray.

In [ ]:
base_colors = {cls: TABLEAU_COLORS[list(TABLEAU_COLORS.keys())[i]]
               for i, cls in enumerate(unique_classes)}

# Function to generate distinct but related colors for Level 2 classifications
def generate_hue_variations(base_color, num_shades):
    """Generate distinct hues within a color family using HSL space."""
    base_rgb = mcolors.to_rgb(base_color)
    base_hls = colorsys.rgb_to_hls(*base_rgb)
    
    # Create variations by adjusting only the HUE component
    hues = np.linspace(base_hls[0] - 0.1, base_hls[0] + 0.1, num_shades)
    hues = np.clip(hues, 0, 1)  # Ensure hues are within valid range
    
    return [mcolors.to_hex(colorsys.hls_to_rgb(hue, base_hls[1], base_hls[2])) for hue in hues]

# Generate Level 2 colors based on hue variations within the Level 1 color family
level_2_groups = level_2_df.groupby("Level_1")["Level_2"].unique()
level_2_colors = {}

for lvl1, lvl2_values in level_2_groups.items():
    shades = generate_hue_variations(base_colors[lvl1], len(lvl2_values))
    for idx, lvl2 in enumerate(lvl2_values):
        level_2_colors[lvl2] = shades[idx]

# Assign colors to nodes based on Level 2 classification
node_colors_level_2 = [
    level_2_colors[G.nodes[n]["level_2"]] if G.nodes[n]["level_2"] in level_2_colors else "#808080"  # Gray color
    for n in G.nodes()
]

# Edge weights for visualization
edge_weights = [G[u][v]['weight'] for u, v in G.edges()]

# Generate graph layout
plt.figure(figsize=(10, 10))
cell_cycle_phases = ["Early G1 Phase", "Late G1 Phase", "S Phase", "G2 Phase", "M Phase"]
legend_handles_lvl2 = [Patch(color=color, label=f'Level 2: {cls}') for cls, color in level_2_colors.items()]
# plt.legend(handles=legend_handles_lvl2, loc="best", fontsize=6, title="Classification Hierarchy")

pos = nx.spring_layout(G, weight='weight', k=0.1, seed=10)

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color=node_colors_level_2, node_size=200, alpha=1, edgecolors="black")

# Draw edges
nx.draw_networkx_edges(G, pos, alpha=0.25, edge_color="black", width=[w * 2 for w in edge_weights])

# Show plot
# plt.title(f"Cho Gene Co-Expression Network (Value-Based, δ = {delta})")
plt.savefig("Network Graphs/cho_gene_coexpression_level2_dataset_level2.png", dpi=300)
plt.axis("off")
plt.show()


# Community Detection Auxiliary Functions

These are the functions used during the Community Detection of our constructed network.

In [ ]:
def visualize_community(G, communities, title, output_dir, file_name, save_file=False):
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G)  # Recalculate layout to ensure all nodes are included

    cmap = plt.get_cmap('viridis', len(communities))

    for i, community in enumerate(communities):
        if community:  # Check if community is not empty
            # Filter out nodes not present in the graph
            valid_nodes = [node for node in community if node in G]
            nx.draw_networkx_nodes(G, pos, nodelist=valid_nodes, node_color=[cmap(i)], alpha=0.8)

    nx.draw_networkx_edges(G, pos, alpha=0.5, edge_color='gray')
    legend_handles = [mpatches.Patch(color=cmap(i), label=f'Community {i + 1}') for i in range(len(communities))]
    plt.legend(handles=legend_handles, loc='upper left', title="Communities")
    plt.title(f"{title} Algorithm Communities")
    plt.axis('off')
    plt.tight_layout()
    if save_file:
        plt.savefig(f"{output_dir}/{file_name}_{title}.png", bbox_inches="tight")
    plt.show()

In [ ]:
def visualize_disconnected_communities(G, communities, color_map, title="Paris Algorithm Communities"):
    """
    Visualizes detected communities as separate disconnected subgraphs while keeping Level 1 colors.

    Parameters:
    - G: NetworkX graph
    - communities: CDLib NodeClustering object (Paris, Infomap, Louvain, etc.)
    - color_map: Dictionary mapping Level 1 categories to colors
    - title: Title for the plot
    """

    # Define layout positioning for separate communities
    num_communities = len(communities.communities)
    fig, ax = plt.subplots(figsize=(12, 8))

    # Generate separate layouts for each community
    layouts = []
    for i in range(num_communities):
        layouts.append(nx.spring_layout(G.subgraph(communities.communities[i]), seed=42))

    # Normalize layout positions to avoid overlap
    spacing = 3
    for i, layout in enumerate(layouts):
        for node in layout:
            layout[node] += [i * spacing, 0]  # Shift positions to separate communities

    # Draw each community separately
    for i, community in enumerate(communities.communities):
        valid_nodes = [node for node in community if node in G]
        if not valid_nodes:
            continue  # Skip empty communities
        subG = G.subgraph(valid_nodes)
        pos = layouts[i]

        # Assign Level 1 colors
        node_colors = [color_map[G.nodes[n]["level_1"]] for n in subG.nodes()]

        # Draw nodes
        nx.draw_networkx_nodes(subG, pos, node_color=node_colors, node_size=200, alpha=1, edgecolors="black")

        # Draw edges
        nx.draw_networkx_edges(subG, pos, alpha=0.3, edge_color="black")

    # Create a legend for Level 1 classification
    cell_cycle_phases = ["Early G1 Phase", "Late G1 Phase", "S Phase", "G2 Phase", "M Phase"]
    legend_handles = [Patch(color=color, label=f'{cell_cycle_phases[cls-1]}') for cls, color in color_map.items()]
    plt.legend(handles=legend_handles, loc="best", fontsize=8, title="Level 1 Classification")

    # Display
    plt.title(title)
    plt.axis("off")
    plt.show()


In [ ]:
import itertools

def modified_rand_score(labels_true, labels_pred, nodes_true_list, nodes_pred_list):
    """
    Compute the Modified Rand Index (MRI), an extension of the Rand Index
    that accounts for missing elements between partitions.

    Parameters
    ----------
    labels_true : list
        True labels corresponding to the nodes.
    labels_pred : list
        Predicted labels corresponding to the nodes.
    nodes_true_list : list
        An ordered list of nodes corresponding to labels_true.
    nodes_pred_list : list
        An ordered list of nodes corresponding to labels_pred.

    Returns
    -------
    mri : float
        The Modified Rand Index score.
    """

    n = len(labels_true)
    total_pairs = n * (n - 1) // 2  # All possible unique node pairs
    n_00, n_11, n_xx = 0, 0, 0  # Initialize counts

    # Convert lists to sets for fast lookup
    nodes_true_set = set(nodes_true_list)
    nodes_pred_set = set(nodes_pred_list)

    # Identify common nodes between the two partitions
    common_nodes = nodes_true_set.intersection(nodes_pred_set)

    # Iterate over all node pairs
    for i, j in itertools.combinations(range(n), 2):  # Iterate over all unique pairs
        node_i, node_j = nodes_true_list[i], nodes_true_list[j]

        # If either node is missing from the common set, count this pair as n_xx.
        if node_i not in common_nodes or node_j not in common_nodes:
            n_xx += 1
        else:
            same_true = (labels_true[i] == labels_true[j])
            same_pred = (labels_pred[i] == labels_pred[j])
            if same_true and same_pred:
                n_11 += 1  # Agreeing pair (same in both)
            elif not same_true and not same_pred:
                n_00 += 1  # Agreeing pair (different in both)

    # Compute MRI Score
    if total_pairs == 0:
        return 1.0  # Perfect match in trivial cases

    MRI = (n_00 + n_11 + n_xx) / total_pairs
    return MRI


# Hierarchical Community Detection

Using our formed network, we employ the Paris and LFM algorithms.

## Paris Algorithm

In [ ]:
paris_communities = algorithms.paris(G)

def evaluate_paris():
	# Compute required metrics
	num_communities = len(paris_communities.communities)
	largest_community_size = max(len(c) for c in paris_communities.communities)
	community_sizes = [len(c) for c in paris_communities.communities]
	num_singletons = sum(1 for c in paris_communities.communities if len(c) == 1)
	modularity_score = evaluation.newman_girvan_modularity(G, paris_communities).score
	community_centralities = []
	for community in paris_communities.communities:
			if len(community) > 1:  # avoid singleton closeness warnings
					subgraph = G.subgraph(community)
					centralities = nx.closeness_centrality(subgraph).values()
					community_centralities.extend(centralities)
	avg_closeness_centrality = np.mean(community_centralities) if community_centralities else 0.0

	predicted_labels_dict = {node: -1 for node in G.nodes()}
	for i, community in enumerate(paris_communities.communities):
			for node in community:
					predicted_labels_dict[node] = i
	predicted_labels = [predicted_labels_dict[node] for node in G.nodes()]

	node_to_community = {node: idx for idx, community in enumerate(paris_communities.communities) for node in community}
	true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]
	# true_labels_level_2 = [G.nodes[n]["level_2"] for n in G.nodes()]

	# ARI calculation
	ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)
	# ari_classifications_score2 = adjusted_rand_score(true_labels_level_2, predicted_labels)

	# RI calculation
	ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)
	# ri_classifications_score2 = rand_score(true_labels_level_2, predicted_labels)

	# MRI calculation
	nodes_true_list = list(G.nodes())
	nodes_pred_list = list(node for community in paris_communities.communities for node in community)
	mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)
	# mri_classifications_score2 = modified_rand_score(true_labels_level_2, predicted_labels, nodes_true_list, nodes_pred_list)

	Paris_results = pd.DataFrame({
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			# "ARI (Level 2)": [ari_classifications_score2],
			"RI (Level 1)": [ri_classifications_score1],
			# "RI (Level 2)": [ri_classifications_score2],
			"MRI (Level 1)": [mri_classifications_score1],
			# "MRI (Level 2)": [mri_classifications_score2],
			"Communities": [paris_communities.communities]
	})
	return Paris_results
Paris_results = evaluate_paris()
# output_dir = "Paris - Graphs"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# Paris_results.to_csv(f"{output_dir}/Paris_Results_Cho.csv")
Paris_results

In [ ]:
visualize_disconnected_communities(G, paris_communities, color_map, title="Level 1 Network: Paris Algorithm Communities")

## LFM Algorithm

In [ ]:
random.seed(42)
np.random.seed(42)

alpha_values = np.arange(0.1, 1.0, 0.1)
lfm_results_list = []
lfm_communities = []
for alpha in alpha_values:
	lfm_alpha = alpha
	curr_community = algorithms.lfm(G, alpha=lfm_alpha)
	lfm_communities.append(curr_community)

	def evaluate_lfm():
		# Compute required metrics
		num_communities = len(curr_community.communities)
		largest_community_size = max(len(c) for c in curr_community.communities)
		community_sizes = [len(c) for c in curr_community.communities]
		num_singletons = sum(1 for c in curr_community.communities if len(c) == 1)
		modularity_score = evaluation.newman_girvan_modularity(G, curr_community).score
		community_centralities = []
		for community in curr_community.communities:
				if len(community) > 1:  # closeness is undefined for single-node graphs
						subgraph = G.subgraph(community)
						centralities = nx.closeness_centrality(subgraph).values()
						community_centralities.extend(centralities)
		avg_closeness_centrality = np.mean(community_centralities)


		node_counts = {}
		for i, community in enumerate(curr_community.communities):
			for node in community:
				if node not in node_counts:
					node_counts[node] = []
				node_counts[node].append(i)

		# Assign each node to the **smallest indexed** community it appears in
		predicted_labels_dict = {node: min(communities) for node, communities in node_counts.items()}

		# Create the final predicted labels list
		predicted_labels = [predicted_labels_dict[n] for n in G.nodes()]

		node_to_community = {node: idx for idx, community in enumerate(curr_community.communities) for node in community}
		true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]

		# ARI calculation
		ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)

		# RI calculation
		ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)

		# MRI calculation
		nodes_true_list = list(G.nodes())
		nodes_pred_list = list(node for community in curr_community.communities for node in community)
		mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)

		LFM_results = pd.DataFrame({
			"Alpha": [lfm_alpha],
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			"RI (Level 1)": [ri_classifications_score1],
			"MRI (Level 1)": [mri_classifications_score1],
			"Communities": [curr_community.communities]
		})
		return LFM_results

	lfm_results_list.append(evaluate_lfm())

# Combine all results into a single DataFrame
LFM_results = pd.concat(lfm_results_list, ignore_index=True)
LFM_results


In [ ]:
# Visualize the best performing LFM alpha value (alpha = 0.7)
best_lfm_communities = lfm_communities[6]
visualize_disconnected_communities(G, best_lfm_communities, color_map, title=f"Level 1 Network: LFM Algorithm Communities (α = {0.7})")

## Girvan Newman Algorithm

In [ ]:
gn_levels = range(1, 11)
gn_results_list = []
gn_communities_list = []

for level in gn_levels:
	gn_level = level
	curr_community = algorithms.girvan_newman(G, level=gn_level)
	gn_communities_list.append(curr_community)

	def evaluate_gn():
		# Compute required metrics
		num_communities = len(curr_community.communities)
		largest_community_size = max(len(c) for c in curr_community.communities)
		community_sizes = [len(c) for c in curr_community.communities]
		num_singletons = sum(1 for c in curr_community.communities if len(c) == 1)
		modularity_score = evaluation.newman_girvan_modularity(G, curr_community).score

		# Per-community closeness
		community_centralities = []
		for community in curr_community.communities:
			if len(community) > 1:
				subgraph = G.subgraph(community)
				centralities = nx.closeness_centrality(subgraph).values()
				community_centralities.extend(centralities)
		avg_closeness_centrality = np.mean(community_centralities)

		# Assign each node to one community
		predicted_labels_dict = {node: -1 for node in G.nodes()}
		for i, community in enumerate(curr_community.communities):
			for node in community:
				predicted_labels_dict[node] = i
		predicted_labels = [predicted_labels_dict[n] for n in G.nodes()]
		true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]

		# Metrics
		ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)
		ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)
		nodes_true_list = list(G.nodes())
		nodes_pred_list = list(node for community in curr_community.communities for node in community)
		mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)

		GN_results = pd.DataFrame({
			"Level": [gn_level],
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			"RI (Level 1)": [ri_classifications_score1],
			"MRI (Level 1)": [mri_classifications_score1],
			"Communities": [curr_community.communities]
		})
		return GN_results

	gn_results_list.append(evaluate_gn())

# Combine all results into a single DataFrame
GN_results = pd.concat(gn_results_list, ignore_index=True)
GN_results


In [ ]:
best_gn_community = gn_communities_list[3]  # Choose the graph with level = 4
visualize_disconnected_communities(G, best_gn_community, color_map, title=f"Level 1 Network: Girvan-Newman Algorithm Communities (GN level = {gn_level})")

# Dynamic Community Detection
Using our formed network, we employ the Infomap and TILES algorithms.

## Infomap Algorithm

In [ ]:
infomap_communities = algorithms.infomap(G)

def evaluate_infomap():
	# Compute required metrics
	num_communities = len(infomap_communities.communities)
	largest_community_size = max(len(c) for c in infomap_communities.communities)
	community_sizes = [len(c) for c in infomap_communities.communities]
	num_singletons = sum(1 for c in infomap_communities.communities if len(c) == 1)
	modularity_score = evaluation.newman_girvan_modularity(G, infomap_communities).score
	community_centralities = []
	for community in curr_community.communities:
		if len(community) > 1:
			subgraph = G.subgraph(community)
			centralities = nx.closeness_centrality(subgraph).values()
			community_centralities.extend(centralities)
	avg_closeness_centrality = np.mean(community_centralities)

	predicted_labels_dict = {node: -1 for node in G.nodes()}
	for i, community in enumerate(infomap_communities.communities):
			for node in community:
					predicted_labels_dict[node] = i
	predicted_labels = [predicted_labels_dict[node] for node in G.nodes()]


	node_to_community = {node: idx for idx, community in enumerate(infomap_communities.communities) for node in community}
	true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]

	# ARI calculation
	ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)

	# RI calculation
	ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)

	# MRI calculation
	nodes_true_list = list(G.nodes())
	nodes_pred_list = list(node for community in infomap_communities.communities for node in community)
	mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)

	Infomap_results = pd.DataFrame({
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			"RI (Level 1)": [ri_classifications_score1],
			"MRI (Level 1)": [mri_classifications_score1],
			"Communities": [infomap_communities.communities]
	})
	return Infomap_results

Infomap_results = evaluate_infomap()
# output_dir = "Infomap - Graphs"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# Infomap_results.to_csv(f"{output_dir}/Infomap_results_Cho.csv")
Infomap_results

In [ ]:
visualize_disconnected_communities(G, infomap_communities, color_map, title="Level 1 Network: Infomap Algorithm Communities")

## Walktrap Algorithm

In [ ]:
walktrap_communities = algorithms.walktrap(G)

def evaluate_walktrap():
	# Compute required metrics
	num_communities = len(walktrap_communities.communities)
	largest_community_size = max(len(c) for c in walktrap_communities.communities)
	community_sizes = [len(c) for c in walktrap_communities.communities]
	num_singletons = sum(1 for c in walktrap_communities.communities if len(c) == 1)
	modularity_score = evaluation.newman_girvan_modularity(G, walktrap_communities).score
	community_centralities = []
	for community in curr_community.communities:
		if len(community) > 1:
			subgraph = G.subgraph(community)
			centralities = nx.closeness_centrality(subgraph).values()
			community_centralities.extend(centralities)
	avg_closeness_centrality = np.mean(community_centralities)

	predicted_labels_dict = {node: -1 for node in G.nodes()}
	for i, community in enumerate(walktrap_communities.communities):
			for node in community:
					predicted_labels_dict[node] = i
	predicted_labels = [predicted_labels_dict[node] for node in G.nodes()]


	node_to_community = {node: idx for idx, community in enumerate(walktrap_communities.communities) for node in community}
	true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]
	# true_labels_level_2 = [G.nodes[n]["level_2"] for n in G.nodes()]

	# ARI calculation
	ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)
	# ari_classifications_score2 = adjusted_rand_score(true_labels_level_2, predicted_labels)

	# RI calculation
	ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)
	# ri_classifications_score2 = rand_score(true_labels_level_2, predicted_labels)

	# MRI calculation
	nodes_true_list = list(G.nodes())
	nodes_pred_list = list(node for community in walktrap_communities.communities for node in community)
	mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)
	# mri_classifications_score2 = modified_rand_score(true_labels_level_2, predicted_labels, nodes_true_list, nodes_pred_list)

	Walktrap_results = pd.DataFrame({
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			# "ARI (Level 2)": [ari_classifications_score2],
			"RI (Level 1)": [ri_classifications_score1],
			# "RI (Level 2)": [ri_classifications_score2],
			"MRI (Level 1)": [mri_classifications_score1],
			# "MRI (Level 2)": [mri_classifications_score2],
			"Communities": [walktrap_communities.communities]
	})
	return Walktrap_results

Walktrap_results = evaluate_walktrap()
# output_dir = "Walktrap - Graphs"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# Walktrap_results.to_csv(f"{output_dir}/Walktrap_results_Cho.csv")
Walktrap_results

In [ ]:
visualize_disconnected_communities(G, walktrap_communities, color_map, title="Level 1 Network: Walktrap Algorithm Communities")

## Spinglass Algorithm

In [ ]:
spinglass_communities = algorithms.spinglass(G)

def evaluate_spinglass():
	# Compute required metrics
	num_communities = len(spinglass_communities.communities)
	largest_community_size = max(len(c) for c in spinglass_communities.communities)
	community_sizes = [len(c) for c in spinglass_communities.communities]
	num_singletons = sum(1 for c in spinglass_communities.communities if len(c) == 1)
	modularity_score = evaluation.newman_girvan_modularity(G, spinglass_communities).score
	community_centralities = []
	for community in curr_community.communities:
		if len(community) > 1:
			subgraph = G.subgraph(community)
			centralities = nx.closeness_centrality(subgraph).values()
			community_centralities.extend(centralities)
	avg_closeness_centrality = np.mean(community_centralities)

	predicted_labels_dict = {node: -1 for node in G.nodes()}
	for i, community in enumerate(spinglass_communities.communities):
			for node in community:
					predicted_labels_dict[node] = i
	predicted_labels = [predicted_labels_dict[node] for node in G.nodes()]


	node_to_community = {node: idx for idx, community in enumerate(spinglass_communities.communities) for node in community}
	true_labels_level_1 = [G.nodes[n]["level_1"] for n in G.nodes()]
	# true_labels_level_2 = [G.nodes[n]["level_2"] for n in G.nodes()]

	# ARI calculation
	ari_classifications_score1 = adjusted_rand_score(true_labels_level_1, predicted_labels)
	# ari_classifications_score2 = adjusted_rand_score(true_labels_level_2, predicted_labels)

	# RI calculation
	ri_classifications_score1 = rand_score(true_labels_level_1, predicted_labels)
	# ri_classifications_score2 = rand_score(true_labels_level_2, predicted_labels)

	# MRI calculation
	nodes_true_list = list(G.nodes())
	nodes_pred_list = list(node for community in spinglass_communities.communities for node in community)
	mri_classifications_score1 = modified_rand_score(true_labels_level_1, predicted_labels, nodes_true_list, nodes_pred_list)
	# mri_classifications_score2 = modified_rand_score(true_labels_level_2, predicted_labels, nodes_true_list, nodes_pred_list)

	Spinglass_results = pd.DataFrame({
			"Number of Communities": [num_communities],
			"Nodes in Largest Community": [largest_community_size],
			"Community Sizes": [community_sizes],
			"Singletons": [num_singletons],
			"Modularity": [modularity_score],
			"Closeness Centrality": [avg_closeness_centrality],
			"ARI (Level 1)": [ari_classifications_score1],
			# "ARI (Level 2)": [ari_classifications_score2],
			"RI (Level 1)": [ri_classifications_score1],
			# "RI (Level 2)": [ri_classifications_score2],
			"MRI (Level 1)": [mri_classifications_score1],
			# "MRI (Level 2)": [mri_classifications_score2],
			"Communities": [spinglass_communities.communities]
	})
	return Spinglass_results

Spinglass_results = evaluate_spinglass()
# output_dir = "Spinglass - Graphs"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# Spinglass_results.to_csv(f"{output_dir}/Spinglass_results_Cho.csv")
Spinglass_results

In [ ]:
visualize_disconnected_communities(G, spinglass_communities, color_map, title="Cho Network: Spinglass Algorithm Communities")

## TILES Algorithm

In [ ]:
print("TILES!")

# Community Detection Results

In [ ]:
Community_Detection_Results = pd.concat([Paris_results, LFM_results.iloc[[6]], GN_results.iloc[[3]], Infomap_results, Walktrap_results], axis=0)
Community_Detection_Results.index = ["Paris", "LFM", "Girvan-Newman", "Infomap", "Walktrap"]

output_dir = "../results/level_1/community_detection_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
Community_Detection_Results.to_csv(f"{output_dir}/level1_cd_results.csv")

Community_Detection_Results